In [1]:
import pandas as pd
import glob
import math
import numpy as np

from helpers import *

# Get the validation files

In [4]:
data_path = 'data/' # TODO: point this to your data folder
#submission_path = data_path + 'submissions/'
training_path = data_path + 'training_set/'
#fin_X = sorted(glob.glob(training_path + "X_val*.csv.gz")) # TODO: point this to a subset of csv's in your training set
#fin_y = sorted(glob.glob(training_path + "y_val*.csv.gz"))
fins = sorted(glob.glob(training_path + "outDD_v2_65.h5"))

In [5]:
fins

['data/training_set/outDD_v2_65.h5']

# Read in the data

In [6]:
df_trn = pd.concat([read_log(f) for f in fins[:1]], axis=0, ignore_index=True)
df_trn['session_id'] = df_trn['session_id'].astype('category')
df_trn.rename({'not_skipped': 'skip_4'}, axis=1, inplace=True)

n_trk=5
X_trn, y_trn, X_trk = get_XY(df_trn, aggs, reset_index=True, 
                  list_musik_qualities_=list_musik_qualities,
                  aggs_music_qualities_=aggs_music_qualities,
                   i_=list(range(n_trk)))

# Get the ground truth

In [7]:
ground_truth = pd.Series(get_y_truth(df_trn))

100%|██████████| 192201/192201 [01:05<00:00, 2955.17it/s]


In [8]:
ground_truth.head()

0             [1, 0, 0, 0, 0, 1, 0]
1    [0, 1, 1, 1, 1, 0, 0, 1, 1, 0]
2    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
3             [1, 0, 1, 0, 0, 0, 0]
4             [0, 0, 0, 0, 0, 0, 1]
dtype: object

# Make predictions using pre-trained models

In [9]:
    l_prob = []
    for i_ in tqdm(list(range(n_trk))):
        X = pd.concat([X_trn, X_trk[i_]], axis=1)

        import pickle
        with open('models/model_v2_m01_{}.pkl'.format(i_), 'rb') as fin:
            mdl = pickle.load(fin)

        prob_pred = mdl.predict_proba(X)[:,1]

        l_prob.append(prob_pred)
        
    preds = pred_series_of_lists([p>0.50 for p in l_prob], ground_truth.apply(len), i_2fill=-2)

100%|██████████| 5/5 [00:03<00:00,  1.32it/s]


In [10]:
preds.head()

0             [1, 1, 1, 1, 0, 0, 0]
1    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0]
2    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3             [1, 1, 1, 1, 1, 1, 1]
4             [0, 0, 0, 0, 0, 0, 0]
dtype: object

# Evaluate the predictions

In [11]:
ap,first_pred_acc = evaluate(preds, ground_truth)

In [12]:
print('average precision: {}'.format(ap))
print('first prediction accuracy: {}'.format(first_pred_acc))

average precision: 0.5570562218283442
first prediction accuracy: 0.7699283562520486


# Do recursive evaluation

In [13]:
for i in range(1,n_trk+1):
    print('------------ {} ----------------'.format(i))
    preds = pred_series_of_lists([l_prob[i_]>0.50 for i_ in range(i)], ground_truth.apply(len), i_2fill=-2)
    ap,first_pred_acc = evaluate(preds, ground_truth)
    print('average precision: {}'.format(ap))
    print('first prediction accuracy: {}'.format(first_pred_acc))

------------ 1 ----------------
average precision: 0.5487262694159871
first prediction accuracy: 0.7699283562520486
------------ 2 ----------------
average precision: 0.5474140530667865
first prediction accuracy: 0.7699283562520486
------------ 3 ----------------
average precision: 0.5508570665227629
first prediction accuracy: 0.7699283562520486
------------ 4 ----------------
average precision: 0.5508776301543791
first prediction accuracy: 0.7699283562520486
------------ 5 ----------------
average precision: 0.5570562218283442
first prediction accuracy: 0.7699283562520486
